In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('groupby_join_app') \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('e:/HCL_Technical_Lead/SIE_OSIV_HCL_TRAINING/PYTHON_DS_DE/Pgadmin_datatalks_sql_DATAPIPELINE/05-batch/data/pq/green/*/*')

In [3]:
df_green.registerTempTable('green')

c:\Users\rajmi\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\sql\dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [5]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [7]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1,2
""")

In [8]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7299999999994|            45|
|2020-01-01 00:00:00|  17|            195.03|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25| 531.0000000000001|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|317.27000000000004|            11|
|2020-01-01 00:00:00|  35|            129.96|             5|
|2020-01-01 00:00:00|  36|295.34000000000003|            11|
|2020-01-01 00:00:00|  37|            175.67|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [12]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('e:/HCL_Technical_Lead/SIE_OSIV_HCL_TRAINING/PYTHON_DS_DE/Pgadmin_datatalks_sql_DATAPIPELINE/05-batch/data/report/revenue/green', mode='overwrite')

In [9]:
df_yellow = spark.read.parquet('e:/HCL_Technical_Lead/SIE_OSIV_HCL_TRAINING/PYTHON_DS_DE/Pgadmin_datatalks_sql_DATAPIPELINE/05-batch/data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

c:\Users\rajmi\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\sql\dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [10]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [11]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-15 11:00:00| 236| 8469.040000000026|           580|
|2020-01-13 19:00:00| 113| 2957.549999999998|           181|
|2020-01-03 21:00:00| 234| 5557.640000000005|           359|
|2020-01-11 00:00:00| 249| 8358.850000000017|           487|
|2020-01-17 17:00:00| 164| 5719.780000000006|           343|
|2020-01-04 04:00:00| 236|            201.16|            12|
|2020-01-08 20:00:00| 144| 2384.079999999999|           133|
|2020-01-21 16:00:00| 231| 4461.609999999999|           205|
|2020-01-31 10:00:00|  68| 4479.939999999999|           256|
|2020-01-06 14:00:00| 261|1979.7199999999993|            90|
|2020-01-02 09:00:00|  48| 4532.439999999997|           260|
|2020-01-21 12:00:00|  50|1343.7999999999997|            79|
|2020-01-21 17:00:00| 107| 5509.840000000002|           314|
|2020-01-29 16:00:00| 23

In [13]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('e:/HCL_Technical_Lead/SIE_OSIV_HCL_TRAINING/PYTHON_DS_DE/Pgadmin_datatalks_sql_DATAPIPELINE/05-batch/data/report/revenue/yellow', mode='overwrite')

In [2]:
df_green_revenue = spark.read.parquet('e:/HCL_Technical_Lead/SIE_OSIV_HCL_TRAINING/PYTHON_DS_DE/Pgadmin_datatalks_sql_DATAPIPELINE/05-batch/data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('e:/HCL_Technical_Lead/SIE_OSIV_HCL_TRAINING/PYTHON_DS_DE/Pgadmin_datatalks_sql_DATAPIPELINE/05-batch/data/report/revenue/yellow')

In [3]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [4]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [5]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   4|              NULL|                NULL|1004.3000000000003|                   57|
|2020-01-01 00:00:00|  10|              NULL|                NULL|             42.41|                    2|
|2020-01-01 00:00:00|  14|              NULL|                NULL|               8.8|                    1|
|2020-01-01 00:00:00|  15|              NULL|                NULL|             34.09|                    1|
|2020-01-01 00:00:00|  22|              15.8|                   1|              NULL|                 NULL|
|2020-01-01 00:00:00|  29|              61.3|                   1|              NULL|                 NULL|
|2020-01-01 00:00:00|  45|  

In [6]:
df_join.write.parquet('e:/HCL_Technical_Lead/SIE_OSIV_HCL_TRAINING/PYTHON_DS_DE/Pgadmin_datatalks_sql_DATAPIPELINE/05-batch/data/report/revenue/total', mode='overwrite')

In [8]:
df_join = spark.read.parquet('e:/HCL_Technical_Lead/SIE_OSIV_HCL_TRAINING/PYTHON_DS_DE/Pgadmin_datatalks_sql_DATAPIPELINE/05-batch/data/report/revenue/total')

In [12]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  24|              87.6|                   3| 754.9500000000003|                   45|
|2020-01-01 00:00:00|  25| 531.0000000000001|                  26|            324.35|                   16|
|2020-01-01 00:00:00|  38| 98.78999999999999|                   2|              NULL|                 NULL|
|2020-01-01 00:00:00|  40|168.98000000000002|                   8|             89.97|                    5|
|2020-01-01 00:00:00|  47|              13.3|                   1|               8.3|                    1|
|2020-01-01 00:00:00|  49|266.76000000000005|                  14|185.64999999999998|                   10|
|2020-01-01 00:00:00|  50|  

In [11]:
df_zones = spark.read.parquet('zones/')

In [13]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [14]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [15]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|  24|              87.6|                   3| 754.9500000000003|                   45|        24|Manhattan|        Bloomingdale| Yellow Zone|
|2020-01-01 00:00:00|  25| 531.0000000000001|                  26|            324.35|                   16|        25| Brooklyn|         Boerum Hill|   Boro Zone|
|2020-01-01 00:00:00|  38| 98.78999999999999|                   2|              NULL|                 NULL|        38|   Queens|     Cambria Heights|   Boro Zone|
|2020-01-01 00:00:00| 

In [16]:
df_result.drop('LocationID', 'zone').show()

+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|               hour|      green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|2020-01-01 00:00:00|              87.6|                   3| 754.9500000000003|                   45|Manhattan| Yellow Zone|
|2020-01-01 00:00:00| 531.0000000000001|                  26|            324.35|                   16| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00| 98.78999999999999|                   2|              NULL|                 NULL|   Queens|   Boro Zone|
|2020-01-01 00:00:00|168.98000000000002|                   8|             89.97|                    5| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|              13.3|                   1|               8.3|                    1|    Bronx|   Bor

In [17]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

In [18]:
spark.stop()
